## random.shuffle 기능을 이용하여 
## training data 와 test data 분리 하는 DataGeneration class 이용

In [1]:
import tensorflow as tf
import numpy as np
import random

In [2]:
class DataGeneration:
    
    def __init__(self, name, file_path, seperation_rate, is_normalized=False):
        
        self.name = name
        
        self.file_path = file_path
        
        self.seperation_rate = seperation_rate
        
        self.is_normalized = is_normalized
        
        print("DataGeneration object is created !!")
    
    # np.max() 를 이용한 데이터 정규화
    def data_normalize_using_max(self, loaded_data):
        
        # 각 열의 최대값을 찾기 위해 행과 열을 바꾸어 줌. 즉 전치향렬을 만들어줌
        transpose_loaded_data = loaded_data.T        

        print("transpose_loaded_data.shape = ", transpose_loaded_data.shape)
        
        # 전치행렬을 위한 리스트
        transpose_normalize_data_list = []

        for index in range(len(transpose_loaded_data)):
    
            max_value = np.max(transpose_loaded_data[index, :])   # 각 행의 최대값을 찾음
    
            # 최대값이 1 이상이면 최대값으로 나누어서 
            # 모든 데이터가 0 ~ 1 사이에 오도록 함
            if max_value > 1.0:  
        
                transpose_normalize_data_list.append(transpose_loaded_data[index, :] / max_value)
       
            # 최대값이 1 보다 작으면 해당 값을 그대로 사용함. 
            ## 왜냐하면 1보다 작은 값이면 굳이 바꿀 필요가 없음
            else:      
        
                transpose_normalize_data_list.append(transpose_loaded_data[index, :])
        

        # 리스트를 numpy type 으로 변환
        transpose_normalize_data = np.array(transpose_normalize_data_list)

        print(transpose_normalize_data.shape)

        # 데이터 저장을 위해 다시 전치행렬을 통해 행과 열을 바꿈
        normalize_data = transpose_normalize_data.T

        print(normalize_data.shape)

        # normalize 파일저장
        data_save_path = './Normalize_' + self.name + '_data.csv'
        
        np.savetxt(data_save_path, normalize_data, delimiter=',')
        
        return normalize_data

    # shuffle 기능을 이용하여 training_data / test_data 생성
    def generate(self):
    
        # 데이터 불러오기

        loaded_data = np.loadtxt(self.file_path, delimiter=',', dtype=np.float32)
        
        print("[debug. before data normalization] loaded_data[0] = ", loaded_data[0])

        print("loaded_data.shape = ", loaded_data.shape)
        
        if (self.is_normalized == True):
            
            loaded_data = self.data_normalize_using_max(loaded_data)
            
            print("[debug. after data normalization] loaded_data[0] = ", loaded_data[0])

        # 임시 저장 리스트
        training_data_list = []
        test_data_list = []

        # 분리비율에 맞게 테스트데이터로 분리
        total_data_num = len(loaded_data)
        test_data_num = int(len(loaded_data) * self.seperation_rate)

        print("total_data_num = ", total_data_num, ", test_data_num = ", test_data_num)

        # 전체 데이터 인덱스를 가지고 있는 리스트 생성
        total_data_index_list = [ index for index in range(total_data_num) ]

        # random.shuffle 을 이용하여 인덱스 리스트 생성
        random.shuffle(total_data_index_list)  # 전체 인덱스가 랜덤하게 섞여진 리스트로 변형된다

        # test data 를 위한 인덱스는 total_data_index_list 로뷰터 앞에서 40 % 의 데이터 인덱스
        test_data_index_list = total_data_index_list[ 0:test_data_num ]

        print("length of test_data_index_list = ", len(test_data_index_list))

        # training data 를 위한 인덱스는 total_data_index_list 에서 test data 인덱스를 제외한 나머지 부분
        training_data_index_list = total_data_index_list[ test_data_num: ]

        print("length of training_data_index_list = ", len(training_data_index_list))

        # training data 구성
        for training_data_index in training_data_index_list:
    
            training_data_list.append(loaded_data[training_data_index])

        # test data 구성
        for test_data_index in test_data_index_list:
    
            test_data_list.append(loaded_data[test_data_index])

        # generate training data from training_data_list using np.arrya(...)
        training_data = np.array(training_data_list)

        # generate test data from test_data_list using np.arrya(...)
        test_data = np.array(test_data_list)

        # verification shape
        print("training_data.shape = ", training_data.shape)
        print("test_data.shape = ", test_data.shape)

        # save training & test data (.csv)
        training_data_save_path = './random_' + self.name + '_training_data.csv'
        test_data_save_path = './random_' + self.name + '_test_data.csv'
        
        np.savetxt(training_data_save_path, training_data, delimiter=',')
        np.savetxt(test_data_save_path, test_data, delimiter=',')
        
        return training_data, test_data

In [3]:
# DataGeneration 객체 생성
seperation_rate = 0.3
data_obj = DataGeneration('Diabetes', './Diabetes.csv', seperation_rate)  # normalize 하지 않음

# training_data, test_data 생성
(training_data, test_data) = data_obj.generate()

print("training_data.shape = ", training_data.shape)
print("test_data.shape = ", test_data.shape)

DataGeneration object is created !!
[debug. before data normalization] loaded_data[0] =  [-0.294118    0.487437    0.180328   -0.292929    0.          0.00149028
 -0.53117    -0.0333333   0.        ]
loaded_data.shape =  (759, 9)
total_data_num =  759 , test_data_num =  227
length of test_data_index_list =  227
length of training_data_index_list =  532
training_data.shape =  (532, 9)
test_data.shape =  (227, 9)
training_data.shape =  (532, 9)
test_data.shape =  (227, 9)


In [4]:
training_x_data = training_data[ :, 0:-1]
training_t_data = training_data[ :, [-1]]

print("training_x_data.shape = ", training_x_data.shape)
print("training_t_data.shape = ", training_t_data.shape)

test_x_data = test_data[ :, 0:-1]
test_t_data = test_data[ :, [-1]]

print("test_x_data.shape = ", test_x_data.shape)
print("test_x_data.shape = ", test_x_data.shape)

training_x_data.shape =  (532, 8)
training_t_data.shape =  (532, 1)
test_x_data.shape =  (227, 8)
test_x_data.shape =  (227, 8)


In [5]:
X = tf.placeholder(tf.float32, [None, 8])  # 8개 입력노드
T = tf.placeholder(tf.float32, [None, 1])  # 1개 정답노드

W = tf.Variable(tf.random_normal([8, 1]))  # 8X1 가중치노드
b = tf.Variable(tf.random_normal([1]))     # 1개 바이어스 노드

In [6]:
z = tf.matmul(X, W) + b  # 선형회귀 값 z

y = tf.sigmoid(z)    # 시그모이드로 계산 값

# 손실함수는 Cross-Entropy 
loss = -tf.reduce_mean( T*tf.log(y) + (1-T)*tf.log(1-y) ) 

In [7]:
learning_rate = 0.01    # 학습율

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)  

In [8]:
# 정확성 검사, True if y > 0.5 else False

# y shape 은 (759X8) (8X1) = 759X1, 즉 759개의 모든 데이터에 대해 y > 0.5 검사함
# 그러므로 predicted 또한 759 개의 1 또는 0 을 가짐
predicted = tf.cast(y > 0.5, dtype=tf.float32)  


# predicted 와 T 같으면 True 를 리턴하므로 cast 에 의해서 1로 강제 변환, 그렇지 않으면 0으로 변환해서
# accuracy 는 총 759 개의 1 또는 0 을 가짐. 이것을 전체 갯수 759로 나눔
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, T), dtype=tf.float32))

In [10]:
with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화

    for step in range(20001):
      
        loss_val, _ = sess.run([loss, train], feed_dict={X: training_x_data, T: training_t_data})    
        
        
        if step % 500 == 0:
            print("step = ", step, ", loss_val = ", loss_val)             
    
    # Accuracy 확인
    y_val, predicted_val, accuracy_val = sess.run([y, predicted, accuracy], feed_dict={X: test_x_data, T: test_t_data})
    
    
    print("\ny_val.shape = ", y_val.shape, ", predicted_val = ", predicted_val.shape)    
    print("\nAccuracy = ", accuracy_val)

step =  0 , loss_val =  0.6965248
step =  500 , loss_val =  0.6352679
step =  1000 , loss_val =  0.6014237
step =  1500 , loss_val =  0.57615703
step =  2000 , loss_val =  0.55715686
step =  2500 , loss_val =  0.5427557
step =  3000 , loss_val =  0.5317255
step =  3500 , loss_val =  0.52317756
step =  4000 , loss_val =  0.5164724
step =  4500 , loss_val =  0.5111499
step =  5000 , loss_val =  0.5068766
step =  5500 , loss_val =  0.5034091
step =  6000 , loss_val =  0.50056773
step =  6500 , loss_val =  0.49821812
step =  7000 , loss_val =  0.49625918
step =  7500 , loss_val =  0.4946134
step =  8000 , loss_val =  0.4932212
step =  8500 , loss_val =  0.492036
step =  9000 , loss_val =  0.4910213
step =  9500 , loss_val =  0.49014753
step =  10000 , loss_val =  0.48939174
step =  10500 , loss_val =  0.4887348
step =  11000 , loss_val =  0.48816136
step =  11500 , loss_val =  0.48765886
step =  12000 , loss_val =  0.48721674
step =  12500 , loss_val =  0.4868264
step =  13000 , loss_val =